In [3]:
# Import libraries 
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import  RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
from pinecone import Pinecone

In [4]:
# Extract data from the pdf
def load_pdf(data):
    loader = DirectoryLoader(data, 
                             glob="*.pdf",
                             loader_cls=PyPDFLoader)
    documents = loader.load()

    return documents



In [5]:
extracted_data = load_pdf('/Users/visakh/Desktop/Gen_AI/medical_chat_bot/Data/')

In [6]:
# Create text chunks 
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=30)
    text_chunk = text_splitter.split_documents(extracted_data)
    return text_chunk


In [7]:
text_chunks = text_split(extracted_data)

In [8]:
len(text_chunks)

40090

In [9]:
import chromadb

In [10]:
!pip show chromadb

Name: chromadb
Version: 0.6.3
Summary: Chroma.
Home-page: https://github.com/chroma-core/chroma
Author: 
Author-email: Jeff Huber <jeff@trychroma.com>, Anton Troynikov <anton@trychroma.com>
License: 
Location: /Users/visakh/Desktop/Gen_AI/medical_chat_bot/.env/lib/python3.12/site-packages
Requires: bcrypt, build, chroma-hnswlib, fastapi, grpcio, httpx, importlib-resources, kubernetes, mmh3, numpy, onnxruntime, opentelemetry-api, opentelemetry-exporter-otlp-proto-grpc, opentelemetry-instrumentation-fastapi, opentelemetry-sdk, orjson, overrides, posthog, pydantic, pypika, PyYAML, rich, tenacity, tokenizers, tqdm, typer, typing_extensions, uvicorn
Required-by: 


In [12]:
chroma_client = chromadb.Client()

In [13]:
collection = chroma_client.create_collection(name="medical_chat_bot")

In [14]:
collection.add(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)

In [16]:
import uuid

In [ ]:
documents = [chunk.page_content for chunk in text_chunks]
ids = [str(uuid.uuid4()) for _ in documents]

collection.add(
    documents=documents,
    ids=ids
)

KeyboardInterrupt: 

: 

In [15]:
 
results = collection.query(
    query_texts=["This is a query document about hawaii"], # Chroma will embed this for you
    n_results=2 # how many results to return
)
print(results)

{'ids': [['id1', 'id2']], 'embeddings': None, 'documents': [['This is a document about pineapple', 'This is a document about oranges']], 'uris': None, 'data': None, 'metadatas': [[None, None]], 'distances': [[1.0403752326965332, 1.2430689334869385]], 'included': [<IncludeEnum.distances: 'distances'>, <IncludeEnum.documents: 'documents'>, <IncludeEnum.metadatas: 'metadatas'>]}
